In [ ]:
!sudo apt-get install libgomp1
!curl -fsSL https://github.com/NexaAI/nexa-sdk/releases/latest/download/nexa-cli_linux_x86_64.sh -o install.sh && chmod +x install.sh && ./install.sh && rm install.sh

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libgomp1 is already the newest version (12.3.0-1ubuntu1~22.04.2).
libgomp1 set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.
>>> Starting Nexa SDK installer...
>>> All required tools are available.
>>> Starting Nexa SDK installation...
>>> Extracting embedded payload to a temporary directory...
>>> Creating installation directory: /opt/nexa_sdk
>>> Installing Nexa SDK files...
>>> Creating symbolic links in /usr/local/bin...
>>> Nexa SDK files installed successfully.
>>> Install complete! The Nexa SDK is now installed.
>>> You can use the 'nexa' commands from your terminal.


In [ ]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
!sudo mv cloudflared-linux-amd64 /usr/local/bin/cloudflared

--2025-12-27 09:51:51--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.11.1/cloudflared-linux-amd64 [following]
--2025-12-27 09:51:51--  https://github.com/cloudflare/cloudflared/releases/download/2025.11.1/cloudflared-linux-amd64
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/106867604/955e9d1b-ac5e-4188-8867-e5f53958a8fe?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-12-27T10%3A46%3A47Z&rscd=attachment%3B+filename%3Dcloudflared-linux-amd64&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-12-27

In [ ]:
!nexa infer NexaAI/Qwen3-VL-2B-Instruct-GGUF


model not found, start download
┃ Choose a quant version to download                                            
┃   F16        [3.2 GiB]                                                        
┃   Q8_0       [1.7 GiB]                                                        
Error: user aborted
Error: download model failed


In [ ]:
import subprocess
import time
import os
import re
import sys

# Configuration
LOCAL_NEXA_URL = "http://127.0.0.1:18181"

def run_combined_service():
    nexa_process = None
    cloudflared_process = None

    try:
        # 1. Start the Nexa server
        print(f"Starting Nexa Server on {LOCAL_NEXA_URL}...")
        nexa_process = subprocess.Popen(
            ["nexa", "serve"],
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            text=True
        )

        # Wait for Nexa to initialize
        # You can adjust this sleep time or implement a health check loop
        time.sleep(10)
        print("Nexa Server should be running.")

        # 2. Start cloudflared tunnel
        print(f"Starting cloudflared tunnel for {LOCAL_NEXA_URL}...")
        cloudflared_process = subprocess.Popen(
            ["cloudflared", "tunnel", "--url", LOCAL_NEXA_URL],
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            text=True
        )

        public_url = None
        print("Waiting for public URL...")

        # 3. Parse cloudflared output for the public URL
        # Use a timeout logic to avoid hanging forever if cloudflared fails
        for line in iter(cloudflared_process.stdout.readline, ''):
            clean_line = line.strip()
            if clean_line:
                print(f"[Cloudflared] {clean_line}")

            # Look for the .trycloudflare.com URL
            match = re.search(r"https://[\w.-]+\.trycloudflare\.com", clean_line)
            if match:
                public_url = match.group(0)
                print("\n" + "="*50)
                print(f"TUNNEL ESTABLISHED!")
                print(f"Public URL: {public_url}")
                print("="*50 + "\n")
                break

        if public_url:
            print("Processes are running. Press Ctrl+C to stop both Nexa and the Tunnel.")
            # Keep the script alive
            while True:
                time.sleep(1)
        else:
            print("Failed to find public URL in cloudflared output.")

    except KeyboardInterrupt:
        print("\nStopping services...")
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        # 4. Cleanup: Shut down both processes
        if cloudflared_process:
            print("Terminating Cloudflared...")
            cloudflared_process.terminate()
            cloudflared_process.wait()

        if nexa_process:
            print("Terminating Nexa Server...")
            nexa_process.terminate()
            nexa_process.wait()

        print("Cleanup complete.")

if __name__ == "__main__":
    run_combined_service()

Starting Nexa Server on http://127.0.0.1:18181...
Nexa Server should be running.
Starting cloudflared tunnel for http://127.0.0.1:18181...
Waiting for public URL...
[Cloudflared] 2025-12-27T09:53:04Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
[Cloudflared] 2025-12-27T09:53:04Z INF Requesting new quick Tunnel on trycloudflare.com...
[Cloudflared] 2025-12-27T09:53:09Z INF +----------------------------------------------------------------------------------------